In [1]:
import os

In [2]:
%pwd

'c:\\Users\\iamsr\\OneDrive\\Desktop\\my_dlproject\\kidney-tumor\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\iamsr\\OneDrive\\Desktop\\my_dlproject\\kidney-tumor'

In [ ]:
# config.yaml
